In [5]:
import numpy as np
#from imageio import imread, imwrite
import cv2

## Naive Method: Increase blue channel

In [8]:
img = cv2.imread('grainger.jpg')

arr = img * np.array([0.5, 0.2, 0.1])
arr2 = (255 * arr/arr.max()).astype(np.uint8)

gamma = 2
gamma_img = np.array(255 * (arr2/255) ** gamma, dtype="uint8")
cv2.imwrite("night_final.png", gamma_img)

True